# Конспект Skillbox
Прошёл обучение на бесплатном интенсиве от Skillbox по программированию на Python. В результате получил такой конспект.

# 1. Преобразование голоса в текст

Телеграм отправляет голосовые в формате .oga, а SpeechRecognition нужен формат .wav.

In [1]:
!pip install SpeechRecognition
!pip install pydub

Функция загрузки файла

In [2]:
import urllib

url = "https://drive.google.com/uc?export=view&id=1aBZnHgsjg7XIVlvpYasOOJ8hurp7V6Ww"
filename = "skillbox_voice_sample.oga"
urllib.request.urlretrieve(url, filename)

('skillbox_voice_sample.oga', <http.client.HTTPMessage at 0x79d639225d50>)

Напишем функцию для преобразования файла:

In [3]:
from pydub import AudioSegment

def oga2wav(filename):
    # Переименование формата: 'sample.oga' -> 'sample.wav'
    new_filename = filename.replace('.oga', '.wav')
    # Читаем файл с диска с помощью функции AudioSegment.from_file()
    audio = AudioSegment.from_file(filename)
    # Экспортируем файл в новом формате
    audio.export(new_filename, format='wav')
    # Возвратим в качестве результата функции имя нового файла
    return new_filename

In [4]:
oga2wav(filename)

'skillbox_voice_sample.wav'

Теперь с помощью библиотеки `speech_recognition` преобразуем голос в текст.

In [5]:
import speech_recognition
import os

recognizer = speech_recognition.Recognizer()

with speech_recognition.WavFile('skillbox_voice_sample.wav') as source:
     wav_audio = recognizer.record(source)

recognizer.recognize_google(wav_audio, language='ru')

'Добрый день Приветствуем всех зрителей skillbox'

Функция преобразования голосового сообщения в текст:

In [6]:
def recognize_speech(oga_filename):
    wav_filename = oga2wav(oga_filename)
    recognizer = speech_recognition.Recognizer()

    with speech_recognition.WavFile(wav_filename) as source:
        wav_audio = recognizer.record(source)

    text = recognizer.recognize_google(wav_audio, language='ru')

    if os.path.exists(oga_filename):
        os.remove(oga_filename)

    if os.path.exists(wav_filename):
        os.remove(wav_filename)

    return text

In [7]:
recognize_speech('skillbox_voice_sample.oga')

'Добрый день Приветствуем всех зрителей skillbox'

Удаление файлов с диска для исключения переполнения:

In [8]:
if os.path.exists('skillbox_voice_sample.oga'):
    os.remove('skillbox_voice_sample.oga')

#проверка условия, есть ли файл

if os.path.exists('skillbox_voice_sample.wav'):
    os.remove('skillbox_voice_sample.wav')


Скачивание файла в телеграм производится с помощью функции-обработчика:
```python
# Новое условие на срабатывание: content_types=['voice']
@bot.message_handler(content_types=['voice'])
def transcript(message):
    # id файла - в message.voice.file_id
    filename = download_file(bot, message.voice.file_id)
    # Распознаем запись с помощью нашей функции recognize_speech
    text = recognize_speech(filename)
    # Отправляем пользователю в ответ текст
    bot.send_message(message.chat.id, text)
```

# 2. Работа с изображениями в библиотеке `Pillow`

Преобразование файла jpg в webp.

In [9]:
from PIL import Image
import glob, os

url = "https://bronk.club/uploads/posts/2023-02/1676926713_bronk-club-p-otkritki-stikeri-krasivo-50.jpg"
filename = "sticker.jpg"
urllib.request.urlretrieve(url, filename)

im = Image.open("sticker.jpg").convert("RGB")
im.save("sticker.webp", "WEBP")

In [10]:
#удаление jpg-изображения
if os.path.exists("sticker.jpg"):
    os.remove("sticker.jpg")

In [11]:
#удаление файла webp
if os.path.exists("sticker.webp"):
    os.remove("sticker.webp")

Наложение фильтра на изображение

In [12]:
# Скачиваем картинку в рабочую папку

import urllib

url = "https://drive.google.com/uc?export=view&id=1clfJFsxixyYEWD4RpBrxOxf9lbE0YbdA"
filename = "test_image.png"
urllib.request.urlretrieve(url, filename)


('test_image.png', <http.client.HTTPMessage at 0x79d6292efb80>)

In [13]:
from PIL import Image, ImageEnhance, ImageFilter

# Считаем изображение
source_image = Image.open('test_image.png')

# Нарисуем исходное изображение
source_image.show()

# Увеличим контрастность
enhanced_image = ImageEnhance.Contrast(source_image).enhance(1.8)

im = enhanced_image.convert("RGB")
im.save("image.png", "PNG")

Наложение фильтров

In [14]:
source_image = Image.open('test_image.png')

# Упражнение: попробуйте различные фильтры:
enhanced_image = source_image.filter(ImageFilter.CONTOUR)
enhanced_image.convert("RGB").save("image.png", "PNG")

Доступные фильтры:
* BLUR
* CONTOUR
* DETAIL
* EDGE_ENHANCE
* EDGE_ENHANCE_MORE
* EMBOSS
* FIND_EDGES
* SHARPEN
* SMOOTH
* SMOOTH_MORE

Наложение текста

In [15]:
from PIL import Image, ImageDraw, ImageFont

im = Image.new('RGB', (200,200), color=('#FAACAC'))
#font = ImageFont.truetype("arial.ttf", size=18)
draw_text = ImageDraw.Draw(im)
draw_text.text(
    (50,100),
    'Help!',
    #font=font,
    fill=('#1C0606')
    )
im.convert("RGB").save("image.jpeg", "JPEG")

Функция для преобразования изображений

In [16]:
def transform_image(filename):
    # Считаем изображение с помощью Image.open
    source_image = Image.open(filename)

    # Наложим фильтр: ImageFilter.EMBOSS
    enhanced_image = source_image.filter(ImageFilter.EMBOSS)

    # Нужно конвертировать RGBA в RGB для сохранения в JPEG:
    enhanced_image = enhanced_image.convert('RGB')

    # Пересохраним изображение:
    enhanced_image.save(filename)

    return filename

Получим функцию обработки сообщения чат-бота с фильтром изображения:  

```python
@bot.message_handler(content_types=['photo'])
def resend_photo(message):
    # Функция отправки обработанного изображения
    file_id = message.photo[-1].file_id
    filename = download_file(bot, file_id)

    # Трансформируем изображение
    transform_image(filename)

    image = open(filename, 'rb')
    bot.send_photo(message.chat.id, image)
    image.close()

    # Не забываем удалять ненужные изображения
    if os.path.exists(filename):


# 3. Парсинг HTML-страниц

Создать файл Excel можно с помощью библиотеки `openpyxl`

In [17]:
from openpyxl import Workbook

Создать таблицу

In [18]:
table = [[1, 2, 3], [4, 5, 6]]
work_book = Workbook()

# Создаем объект "листа"
work_sheet = work_book.active

# Коллекция для генерации таблицы
numbers = [1, 2, 3, 4, 5]

for elem in numbers:
    row = [elem, elem * 2, elem ** 2]
    print(row)
    work_sheet.append(row)

work_book.save('table.xlsx')

[1, 2, 1]
[2, 4, 4]
[3, 6, 9]
[4, 8, 16]
[5, 10, 25]


In [19]:
import requests

web_page = requests.get('https://live.skillbox.ru/playlists/code/python/')

web_page.text

'<!doctype html>\n<html data-n-head-ssr lang="ru" data-n-head="%7B%22lang%22:%7B%22ssr%22:%22ru%22%7D%7D">\n  <head >\n    <meta data-n-head="ssr" charset="utf-8"><meta data-n-head="ssr" name="viewport" content="width=device-width, initial-scale=1"><meta data-n-head="ssr" name="referrer" content="no-referrer-when-downgrade"><meta data-n-head="ssr" data-hid="og:type" property="og:type" content="site"><meta data-n-head="ssr" data-hid="og:site_name" property="og:site_name" content="live.skillbox.ru"><meta data-n-head="ssr" data-hid="og:locale" property="og:locale" content="ru_RU"><meta data-n-head="ssr" data-hid="twitter:card" property="twitter:card" content="summary_large_image"><meta data-n-head="ssr" data-hid="vk:image" property="vk:image" content="https://cdn.skillbox.pro/wbd-front/live-static/general/og/1074×480.png"><meta data-n-head="ssr" data-hid="twitter:image" property="twitter:image" content="https://cdn.skillbox.pro/wbd-front/live-static/general/og/1200×630.png"><meta data-n-h

Вызов парсера

In [20]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(web_page.text, 'html.parser')


```html
   <h2 class="playlist-inner-card__link-text">
    Пишем интерфейс мессенджера на Python
   </h2>
```

Ищем нужную запись по имени класса `(class_='...')` и извлекаем текст (`.text`)

In [21]:
soup.find(class_='playlist-inner-card__link-text').text

'Пишем интерфейс мессенджера на Python'

Найти ссылку на видео:
```html
<a class="playlist-inner-card__link" href="/webinars/code/pishem-interfeis-messendzhera-na-python150323/">
```

Ищем нужную запись по имени класса и из атрибутов `.attrs` извлекаем атрибут `'href'`

In [22]:
relative_url = soup.find(class_='playlist-inner-card__link').attrs['href']

abs_url = 'https://live.skillbox.ru' + relative_url

abs_url

'https://live.skillbox.ru/webinars/code/pishem-interfeis-messendzhera-na-python060523/'

Переносим информацию в таблицу

In [23]:
work_book = Workbook()
work_sheet = work_book.active

items = soup.find_all(class_='playlist-inner__item')

for elem in items:
    title = elem.find(class_='playlist-inner-card__link-text').text
    relative_url = elem.find(class_='playlist-inner-card__link').attrs['href']
    video = elem.find(class_='playlist-inner-card__small-info').text[8:23]
    url = 'https://live.skillbox.ru' + relative_url
    row = [title, url, video]
    print(row)
    work_sheet.append(row)

work_book.save('table.xlsx')

['Пишем интерфейс мессенджера на Python', 'https://live.skillbox.ru/webinars/code/pishem-interfeis-messendzhera-na-python060523/', 'Видео, 01:51:00']
['Настраиваем протокол TCP для мессенджера', 'https://live.skillbox.ru/webinars/code/nastraivaem-protokol-tcp-dlya-messendzhera050523/', 'Видео, 01:58:13']
['Знакомимся с Python', 'https://live.skillbox.ru/webinars/code/znakomimsya-s-python040523/', 'Видео, 01:47:47']
['Искусственный интеллект на Python: подводим итоги', 'https://live.skillbox.ru/webinars/code/iskusstvennyi-intellekt-na-python-podvodim-itogi211222/', 'Видео, 01:04:56']
['Учим бота на Python понимать текст', 'https://live.skillbox.ru/webinars/code/uchim-bota-na-python-ponimat-tekst201222/', 'Видео, 01:53:26']
['Знакомимся с Python и архитектурой умного чат-бота', 'https://live.skillbox.ru/webinars/code/znakomimsya-s-python-i-arkhitekturoi-umnogo-chat-bota191222/', 'Видео, 02:10:05']
['Хакинг на Python: пишем первое приложение', 'https://live.skillbox.ru/webinars/code/khaki

# 4. Веб-страница на фреймворке Flask

In [24]:
from flask import Flask, render_template

app = Flask(__name__)

@app.route('/')
def hello_world():
   return '<h1>Hello from Skillbox!<h1>'

@app.route('/hello/')
@app.route('/hello/<name>')
def hello(name=None):
   return render_template('hello.html', name=name)

Содержание файла hello.html


```html
<!doctype html>
<title>Hello from Skillbox</title>
{% if name %}
  <h1>Hello {{ name }}!</h1>
{% else %}
  <h1>Hello, World!</h1>
{% endif %}
```
